## Introduction

We will be working with a data set based on [housing prices in Ames, Iowa](https://www.kaggle.com/c/house-prices-advanced-regression-techniques?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork783-2023-01-01). It was compiled for educational use to be a modernized and expanded alternative to the well-known Boston Housing dataset. This version of the data set has had some missing values filled for convenience.

There are an extensive number of features, so they've been described in the table below.

### Predictor

* SalePrice: The property's sale price in dollars. 

### Features

* MoSold: Month Sold
* YrSold: Year Sold   
* SaleType: Type of sale
* SaleCondition: Condition of sale
* MSSubClass: The building class
* MSZoning: The general zoning classification
* ...


In [68]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [69]:
import piplite
await piplite.install(['tqdm', 'seaborn', 'skillsnetwork', 'pandas', 'numpy', 'scikit-learn'])

## Question 1

* Import the data using Pandas and examine the shape. There are 79 feature columns plus the predictor, the sale price (`SalePrice`). 
* There are three different types: integers (`int64`), floats (`float64`), and strings (`object`, categoricals). Examine how many there are of each data type. 


In [70]:
import pandas as pd
import numpy as np
import skillsnetwork 

data = pd.read_csv('Ames_Housing_Sales.csv')
data.head()

print(data.shape)

(1379, 80)


In [71]:
data.dtypes.value_counts()

object     43
float64    21
int64      16
dtype: int64

In [72]:
def is_string(x):
    return isinstance(x, str)

categorical_data = data.select_dtypes(exclude="number")

is_string_series = categorical_data[categorical_data.columns].apply(is_string)
# if is_string_series.any():
#     print('The column contains string data')
# else:
#     print('The column does not contain string data')
is_string_series
categorical_data

,Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,CentralAir,Condition1,Condition2,...,MiscFeature,Neighborhood,PavedDrive,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,Street,Utilities
0,None,1Fam,TA,No,GLQ,Unf,Gd,Y,Norm,Norm,...,None,CollgCr,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
1,None,1Fam,TA,Gd,ALQ,Unf,Gd,Y,Feedr,Norm,...,None,Veenker,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
2,None,1Fam,TA,Mn,GLQ,Unf,Gd,Y,Norm,Norm,...,None,CollgCr,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
3,None,1Fam,Gd,No,ALQ,Unf,TA,Y,Norm,Norm,...,None,Crawfor,Y,None,CompShg,Gable,Abnorml,WD,Pave,AllPub
4,None,1Fam,TA,Av,GLQ,Unf,Gd,Y,Norm,Norm,...,None,NoRidge,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,None,1Fam,None,None,None,None,None,Y,Norm,Norm,...,None,Gilbert,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
1375,None,1Fam,TA,No,ALQ,Rec,Gd,Y,Norm,Norm,...,None,NWAmes,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
1376,None,1Fam,Gd,No,GLQ,Unf,TA,Y,Norm,Norm,...,Shed,Crawfor,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
1377,None,1Fam,TA,Mn,GLQ,Rec,TA,Y,Norm,Norm,...,None,NAmes,Y,None,CompShg,Hip,Normal,WD,Pave,AllPub


## Question 2

A significant challenge, particularly when dealing with data that have many columns, is ensuring each column gets encoded correctly. 

This is particularly true with data columns that are ordered categoricals (ordinals) vs unordered categoricals. Unordered categoricals should be one-hot encoded, however this can significantly increase the number of features and creates features that are highly correlated with each other.

Determine how many total features would be present, relative to what currently exists, if all string (object) features are one-hot encoded. Recall that the total number of one-hot encoded columns is `n-1`, where `n` is the number of categories.


In [73]:
# numerical_columns = data.select_dtypes(include="number")

# categorical_columns.Alley.value_counts()

# one_hot_encoded_data = pd.get_dummies(categorical_columns, columns = categorical_columns.columns)

# one_hot_encoded_data.shape[0]
# # numerical_columns.shape[0]
# # categorical_columns

# from sklearn.preprocessing import OneHotEncoder
  
# # Retrieving data
categorical_data = data.select_dtypes(exclude="number")
numerical_data = data.select_dtypes(include="number")
one_hot_encoded_data = pd.get_dummies(categorical_data, columns = categorical_data.columns)
new_no_features  = one_hot_encoded_data.shape[1] + numerical_data.shape[1]
old_no_features = data.shape[1]
new_no_features
old_no_features

print("There are {} new features".format(new_no_features-old_no_features))

There are 215 new features


## Question 3

Let's create a new data set where all of the above categorical features will be one-hot encoded. We can fit this data and see how it affects the results.

* Used the dataframe `.copy()` method to create a completely separate copy of the dataframe for one-hot encoding
* On this new dataframe, one-hot encode each of the appropriate columns and add it back to the dataframe. Be sure to drop the original column.
* For the data that are not one-hot encoded, drop the columns that are string categoricals.

For the first step, numerically encoding the string categoricals, either Scikit-learn;s `LabelEncoder` or `DictVectorizer` can be used. However, the former is probably easier since it doesn't require specifying a numerical value for each category, and we are going to one-hot encode all of the numerical values anyway. (Can you think of a time when `DictVectorizer` might be preferred?)


In [74]:
categorical_copy = categorical_data.copy()


In [75]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for col in categorical_copy:
    categorical_copy[col+"_encoded"] = label_encoder.fit_transform(categorical_copy[col])



In [76]:
categorical_copy.head(20)

,Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,CentralAir,Condition1,Condition2,...,MiscFeature_encoded,Neighborhood_encoded,PavedDrive_encoded,PoolQC_encoded,RoofMatl_encoded,RoofStyle_encoded,SaleCondition_encoded,SaleType_encoded,Street_encoded,Utilities_encoded
0,None,1Fam,TA,No,GLQ,Unf,Gd,Y,Norm,Norm,...,1,5,2,3,1,1,4,8,1,0
1,None,1Fam,TA,Gd,ALQ,Unf,Gd,Y,Feedr,Norm,...,1,24,2,3,1,1,4,8,1,0
2,None,1Fam,TA,Mn,GLQ,Unf,Gd,Y,Norm,Norm,...,1,5,2,3,1,1,4,8,1,0
3,None,1Fam,Gd,No,ALQ,Unf,TA,Y,Norm,Norm,...,1,6,2,3,1,1,0,8,1,0
4,None,1Fam,TA,Av,GLQ,Unf,Gd,Y,Norm,Norm,...,1,15,2,3,1,1,4,8,1,0
5,None,1Fam,TA,No,GLQ,Unf,Gd,Y,Norm,Norm,...,3,11,2,3,1,1,4,8,1,0
6,None,1Fam,TA,Av,GLQ,Unf,Ex,Y,Norm,Norm,...,1,21,2,3,1,1,4,8,1,0
7,None,1Fam,TA,Mn,ALQ,BLQ,Gd,Y,PosN,Norm,...,3,14,2,3,1,1,4,8,1,0
8,None,1Fam,None,None,None,None,None,Y,Artery,Norm,...,1,17,2,3,1,1,0,8,1,0
9,None,2fmCon,TA,No,GLQ,Unf,TA,Y,Artery,Artery,...,1,3,2,3,1,1,4,8,1,0


In [77]:
categorical_copy = categorical_copy.drop(categorical_data.columns, axis=1)

In [78]:
categorical_copy.head(20)
numerical_data.head(20)

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,OverallQual,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SalePrice
0,856.0,854.0,0.0,3,706.0,0.0,1,0,150.0,0.0,...,7,0.0,0.0,8,856.0,0.0,2003,2003,2008,208500.0
1,1262.0,0.0,0.0,3,978.0,0.0,0,1,284.0,0.0,...,6,0.0,0.0,6,1262.0,298.0,1976,1976,2007,181500.0
2,920.0,866.0,0.0,3,486.0,0.0,1,0,434.0,0.0,...,7,0.0,0.0,6,920.0,0.0,2001,2002,2008,223500.0
3,961.0,756.0,0.0,3,216.0,0.0,1,0,540.0,272.0,...,7,0.0,0.0,7,756.0,0.0,1915,1970,2006,140000.0
4,1145.0,1053.0,0.0,4,655.0,0.0,1,0,490.0,0.0,...,8,0.0,0.0,9,1145.0,192.0,2000,2000,2008,250000.0
5,796.0,566.0,320.0,1,732.0,0.0,1,0,64.0,0.0,...,5,0.0,0.0,5,796.0,40.0,1993,1995,2009,143000.0
6,1694.0,0.0,0.0,3,1369.0,0.0,1,0,317.0,0.0,...,8,0.0,0.0,7,1686.0,255.0,2004,2005,2007,307000.0
7,1107.0,983.0,0.0,3,859.0,32.0,1,0,216.0,228.0,...,7,0.0,0.0,7,1107.0,235.0,1973,1973,2009,200000.0
8,1022.0,752.0,0.0,2,0.0,0.0,0,0,952.0,205.0,...,7,0.0,0.0,8,952.0,90.0,1931,1950,2008,129900.0
9,1077.0,0.0,0.0,2,851.0,0.0,1,0,140.0,0.0,...,5,0.0,0.0,5,991.0,0.0,1939,1950,2008,118000.0


In [79]:
encoded_data= pd.concat([categorical_copy, numerical_data], axis=1, join='inner')
encoded_data.head()

,Alley_encoded,BldgType_encoded,BsmtCond_encoded,BsmtExposure_encoded,BsmtFinType1_encoded,BsmtFinType2_encoded,BsmtQual_encoded,CentralAir_encoded,Condition1_encoded,Condition2_encoded,...,OverallQual,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SalePrice
0,1,0,3,3,2,6,2,1,2,2,...,7,0.0,0.0,8,856.0,0.0,2003,2003,2008,208500.0
1,1,0,3,1,0,6,2,1,1,2,...,6,0.0,0.0,6,1262.0,298.0,1976,1976,2007,181500.0
2,1,0,3,2,2,6,2,1,2,2,...,7,0.0,0.0,6,920.0,0.0,2001,2002,2008,223500.0
3,1,0,1,3,0,6,4,1,2,2,...,7,0.0,0.0,7,756.0,0.0,1915,1970,2006,140000.0
4,1,0,3,0,2,6,2,1,2,2,...,8,0.0,0.0,9,1145.0,192.0,2000,2000,2008,250000.0


## Question 4

* Create train and test splits of both data sets. To ensure the data gets split the same way, use the same `random_state` in each of the two splits.
* For each data set, fit a basic linear regression model on the training data. 
* Calculate the mean squared error on both the train and test sets for the respective models. Which model produces smaller error on the test data and why?


In [80]:

target = data["SalePrice"]
encoded_train = numerical_data.drop("SalePrice",axis=1)
train = numerical_data.drop("SalePrice",axis=1)


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

encoded_X_train, encoded_X_test, encoded_y_train, encoded_y_test = train_test_split(encoded_train, target, random_state=43, test_size=0.3)
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state =43,test_size=0.3 )

X_train.shape

(965, 36)

In [82]:
from sklearn.linear_model import LinearRegression

encoded_model = LinearRegression()
model = LinearRegression()

encoded_model.fit(encoded_X_train,encoded_y_train)
model.fit(X_train, y_train)

LinearRegression()

In [83]:
predict_test_enc = encoded_model.predict(encoded_X_test)
predict_test = model.predict(X_test)

predict_train_enc = encoded_model.predict(encoded_X_train)
predict_train = model.predict(X_train)

In [84]:
encoded_train_error = mean_squared_error(encoded_y_train, predict_train_enc)
encoded_test_error = mean_squared_error(encoded_y_test, predict_test_enc)

In [85]:
encoded_train_error

1015933269.8261256